<a href="https://colab.research.google.com/github/mojtabaSefidi/Fall-Detection-System/blob/main/Fall_Detection_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import libraries and frameworks

In [1]:
from sklearn import preprocessing
import tensorflow as tf
from math import sqrt
import pandas as pd
import numpy as np
import glob
import os
window_size = 200


Get the sisfall and sisfall_enhanced dataset

In [ ]:
!gdown --id 1kyTRhIFhqwRkf9gERof1Xm5FVQ-klLVA
!gdown --id 1gvOuxPc8dNgTnxuvPcVuCKifOf98-TV0
!unzip SisFall_dataset.zip
!unzip SisFall_enhanced.zip

# Implementation

Get all addresses

In [3]:
def get_file_name(path):
  allfiles = []
  allFolders = glob.glob(path + "*")
  for files in allFolders:
      allfiles.append(glob.glob(files+"/*.txt"))
  if 'desktop.ini' in allfiles:
        allfiles.remove('desktop.ini')
  return np.hstack(allfiles)

In [4]:
get_file_name("SisFall_dataset/")

array(['SisFall_dataset/SA05/D15_SA05_R05.txt',
       'SisFall_dataset/SA05/F01_SA05_R01.txt',
       'SisFall_dataset/SA05/D15_SA05_R04.txt', ...,
       'SisFall_dataset/SE15/D08_SE15_R01.txt',
       'SisFall_dataset/SE15/D05_SE15_R02.txt',
       'SisFall_dataset/SE15/D14_SE15_R01.txt'], dtype='<U37')

read dataset from address path

In [5]:
def read_data(data_path):
    data = pd.read_csv(data_path, header=None)
    data.columns = ['ADXL345_x', 'ADXL345_y', 'ADXL345_z', 'ITG3200_x', 'ITG3200_y', 'ITG3200_z', 'MMA8451Q_x',
                    'MMA8451Q_y', 'MMA8451Q_z']
    data['MMA8451Q_z'] = data['MMA8451Q_z'].map(lambda x: str(x)[:-1])
    for name in data.columns :
      data[name] = data[name].astype(float)
    return data

Feature Extraction

In [6]:
def add_features(dataset,data_path):
    new_dataset = pd.DataFrame()
    new_dataset['acc_1'] = dataset.apply(
        lambda row: sqrt((row.ADXL345_x ** 2 + row.ADXL345_y ** 2 + row.ADXL345_z ** 2)), axis=1)
    new_dataset['acc_2'] = dataset.apply(
        lambda row: sqrt((row.MMA8451Q_x ** 2 + row.MMA8451Q_y ** 2 + row.MMA8451Q_z ** 2)), axis=1)
    new_dataset['geo'] = dataset.apply(
        lambda row: sqrt((row.ITG3200_x ** 2 + row.ITG3200_y ** 2 + row.ITG3200_z ** 2)), axis=1)
    new_dataset['label'] = get_label(data_path)
    return np.round(new_dataset.to_numpy(),2)


Get the Label

In [7]:
def get_label(data_path):
    label = data_path[21]
    if label =='D':
      return int(0)
    elif label =='F':  
      label_path = data_path.replace('dataset','enhanced')
      labels = pd.read_csv(label_path,header=None)
      labels[labels == 2] = 1
      return labels


Split Dataset to train and test

In [8]:
def split_address(dataset_address):
  np.random.shuffle(dataset_address)
  train, test = np.split(dataset_address, [int(len(dataset_address)*0.7)])
  return train, test

Extract features from All addresses

In [9]:
def datasets_to_nparray(datasets_address_array):
  result = np.empty((0, 4), int)
  for address in datasets_address_array:
    result = np.concatenate(
        (result,add_features(read_data(address),address)),axis=0)
  return result

Windowing of the dataset

In [10]:
def windowing(dataset,window_size):
  window = window_size * (dataset.shape[1]-1)
  cut = dataset.shape[0] % window_size
  feature = dataset[:-cut,0:-1]
  label = dataset[:-cut,-1]
  feature = feature.ravel().reshape(feature.size//window,window)
  label = label.reshape(label.size//window_size,window_size)
  label = label.sum(axis=1)
  label[label > 0] = 1
  return feature,label

Save the Train and Test Dataset as Tensor

In [11]:
def dataset_to_tensor(test,train,window_size):
  test_feature , test_label = windowing(datasets_to_nparray(test),window_size)
  np.savez('Sisfall_data_test', inputs=test_feature, targets=test_label)
  train_feature , train_label = windowing(datasets_to_nparray(train),window_size)
  np.savez('Sisfall_data_train', inputs=train_feature, targets=train_label)


# Main

Data Pre-processing

In [12]:
train, test = split_address(get_file_name("SisFall_dataset/"))

In [13]:
dataset_to_tensor(test,train,window_size)

In [14]:
npz = np.load("Sisfall_data_train.npz")
train_inputs = preprocessing.scale(npz["inputs"].astype(np.float))
train_targets = npz["targets"].astype(np.int)


npz = np.load("Sisfall_data_test.npz")
test_inputs = preprocessing.scale(npz["inputs"].astype(np.float))
test_targets = npz["targets"].astype(np.int)

In [15]:
print(train_inputs.shape)
print(train_targets.shape)
print(test_inputs.shape)
print(test_targets.shape)

(55792, 600)
(55792,)
(23502, 600)
(23502,)


Neural Network Training

In [16]:
input_size = 600
output_size = 1
hidden_layer_size = 50

model = tf.keras.Sequential([
                             tf.keras.layers.Dense(input_size,activation="relu"),
                             tf.keras.layers.Dense(hidden_layer_size, activation="relu"),
                             tf.keras.layers.Dense(hidden_layer_size,activation="relu"),
                             tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

batch_size = 100
max_epochs = 50

model.fit(train_inputs,
          train_targets,
          batch_size=batch_size,
          epochs=max_epochs,
          shuffle = True,
          verbose = 1
          )

Epoch 1/50
558/558 [==============================] - 4s 5ms/step - loss: 0.1004 - accuracy: 0.9690
Epoch 2/50
558/558 [==============================] - 3s 5ms/step - loss: 0.0694 - accuracy: 0.9766
Epoch 3/50
558/558 [==============================] - 3s 5ms/step - loss: 0.0596 - accuracy: 0.9795
Epoch 4/50
558/558 [==============================] - 3s 5ms/step - loss: 0.0568 - accuracy: 0.9803
Epoch 5/50
558/558 [==============================] - 3s 5ms/step - loss: 0.0545 - accuracy: 0.9817
Epoch 6/50
558/558 [==============================] - 3s 5ms/step - loss: 0.0516 - accuracy: 0.9819
Epoch 7/50
558/558 [==============================] - 3s 5ms/step - loss: 0.0481 - accuracy: 0.9829
Epoch 8/50
558/558 [==============================] - 3s 5ms/step - loss: 0.0458 - accuracy: 0.9842
Epoch 9/50
558/558 [==============================] - 3s 5ms/step - loss: 0.0452 - accuracy: 0.9843
Epoch 10/50
558/558 [==============================] - 3s 5ms/step - loss: 0.0452 - accuracy: 0.9841

Neural Network Evaluation

In [17]:
loss, accuracy = model.evaluate(test_inputs, test_targets, verbose=1)

735/735 [==============================] - 1s 1ms/step - loss: 0.1006 - accuracy: 0.9801


Confusion Matrix

In [21]:
predictions = model.predict(test_inputs)
conf_matrix = tf.math.confusion_matrix(labels=test_targets,
                                       predictions=predictions).numpy()
print("------ Confusion Matrix Created ------")
print()
print(conf_matrix)
print()

precision_1 = (conf_matrix[1][1]/(conf_matrix[1][1]+conf_matrix[0][1])).round(2)
recall_1 = (conf_matrix[1][1]/(conf_matrix[1][1]+conf_matrix[1][0])).round(2)
f1Score_1 = ((2 * precision_1 * recall_1)/(precision_1 + recall_1)).round(2)
statics = pd.DataFrame([[precision_1,recall_1,f1Score_1]],columns=["Precision","Recall","F1score"])
statics

------ Confusion Matrix Created ------

[[22397     6]
 [  732   367]]



,Precision,Recall,F1score
0,0.98,0.33,0.49


#### SVM and Logistic Regression

In [22]:
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC
x = train_inputs
y = train_targets
x_test = test_inputs
y_test = test_targets

Logistic Regression

In [23]:
reg = LogisticRegression()
reg.fit(x,y)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

Logistic Regression Evaluation

In [24]:
print("score on test: " + str(reg.score(x_test, y_test)))
print("score on train: "+ str(reg.score(x, y)))
print()
y_pred = reg.predict(x_test)
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

score on test: 0.9540464641307123
score on train: 0.9601376541439632

              precision    recall  f1-score   support

           0       0.96      0.99      0.98     22403
           1       0.53      0.17      0.26      1099

    accuracy                           0.95     23502
   macro avg       0.74      0.58      0.62     23502
weighted avg       0.94      0.95      0.94     23502

[[22230   173]
 [  907   192]]


Svm Model


In [25]:
svm=LinearSVC(C = 0.0001)
svm.fit(x, y)


LinearSVC(C=0.0001, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0)

SVM Model Evaluation

In [26]:
print("score on test: "  + str(svm.score(x_test, y_test)))
print("score on train: " + str(svm.score(x, y)))
print()
y_pred = svm.predict(x_test)
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

score on test: 0.9528125265934814
score on train: 0.9549397763120161

              precision    recall  f1-score   support

           0       0.95      1.00      0.98     22403
           1       0.38      0.01      0.03      1099

    accuracy                           0.95     23502
   macro avg       0.67      0.51      0.50     23502
weighted avg       0.93      0.95      0.93     23502

[[22377    26]
 [ 1083    16]]


KNN implementation

In [27]:
knn = KNeighborsClassifier(n_neighbors=4)
knn.fit(x, y)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=4, p=2,
                     weights='uniform')

KNN evaluation

In [28]:
y_pred = knn.predict(x_test)
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.98      1.00      0.99     22403
           1       0.94      0.56      0.70      1099

    accuracy                           0.98     23502
   macro avg       0.96      0.78      0.85     23502
weighted avg       0.98      0.98      0.98     23502

[[22362    41]
 [  479   620]]


Balancing the Dataset

In [29]:
new_train_dataset = np.concatenate((train_inputs,train_targets.reshape((train_targets.shape[0],1))),axis=1)
df = pd.DataFrame(new_train_dataset, columns = None)
df.columns = [*df.columns[:-1], 'label']
reps = [305 if val == 1 else 1 for val in df.label]
df = df.loc[np.repeat(df.index.values, reps)].reset_index(drop=True)
Balanced_train = df.to_numpy()
print('Shape before Balancing :', new_train_dataset.shape)
print('Shape After Balancing:', Balanced_train.shape)

Shape before Balancing : (55792, 601)
Shape After Balancing: (829776, 601)


Train the model with Balanced dataset

In [30]:
input_size = 600
output_size = 1
hidden_layer_size = 50

model = tf.keras.Sequential([
                             tf.keras.layers.Dense(input_size,activation="relu"),
                             tf.keras.layers.Dense(hidden_layer_size, activation="relu"),
                             tf.keras.layers.Dense(hidden_layer_size,activation="relu"),
                             tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

batch_size = 100
max_epochs = 50

model.fit(Balanced_train[:,:-1],
          Balanced_train[:,-1],
          batch_size=batch_size,
          epochs=max_epochs,
          shuffle = True,
          verbose = 1
          )

Epoch 1/50
8298/8298 [==============================] - 41s 5ms/step - loss: 0.0470 - accuracy: 0.9859
Epoch 2/50
8298/8298 [==============================] - 40s 5ms/step - loss: 0.0258 - accuracy: 0.9930
Epoch 3/50
8298/8298 [==============================] - 40s 5ms/step - loss: 0.0195 - accuracy: 0.9948
Epoch 4/50
8298/8298 [==============================] - 41s 5ms/step - loss: 0.0156 - accuracy: 0.9960
Epoch 5/50
8298/8298 [==============================] - 39s 5ms/step - loss: 0.0131 - accuracy: 0.9967
Epoch 6/50
8298/8298 [==============================] - 39s 5ms/step - loss: 0.0106 - accuracy: 0.9973
Epoch 7/50
8298/8298 [==============================] - 39s 5ms/step - loss: 0.0090 - accuracy: 0.9977
Epoch 8/50
8298/8298 [==============================] - 40s 5ms/step - loss: 0.0075 - accuracy: 0.9981
Epoch 9/50
8298/8298 [==============================] - 39s 5ms/step - loss: 0.0071 - accuracy: 0.9982
Epoch 10/50
8298/8298 [==============================] - 40s 5ms/step - l

Testing the model

In [31]:
loss, accuracy = model.evaluate(test_inputs, test_targets, verbose=1)


735/735 [==============================] - 1s 2ms/step - loss: 0.3746 - accuracy: 0.9760


Evaluation of the model

In [32]:
predictions = model.predict(test_inputs)
conf_matrix = tf.math.confusion_matrix(labels=test_targets,
                                       predictions=predictions).numpy()
print("------ Confusion Matrix Created ------")
print()
print(conf_matrix)
print()

precision_2 = (conf_matrix[1][1]/(conf_matrix[1][1]+conf_matrix[0][1])).round(2)
recall_2 = (conf_matrix[1][1]/(conf_matrix[1][1]+conf_matrix[1][0])).round(2)
f1Score_2 = ((2 * precision_2 * recall_2)/(precision_2 + recall_2)).round(2)
statics = pd.DataFrame([[precision_2,recall_2,f1Score_2]],columns=["Precision","Recall","F1score"])
statics

------ Confusion Matrix Created ------

[[22314    89]
 [  438   661]]



,Precision,Recall,F1score
0,0.88,0.6,0.71


Balancing the train dataset for Svm and Logistic Regression Model

In [33]:
new_train_dataset = np.concatenate((train_inputs,train_targets.reshape((train_targets.shape[0],1))),axis=1)
df = pd.DataFrame(new_train_dataset, columns = None)
df.columns = [*df.columns[:-1], 'label']
reps = [128 if val == 1 else 1 for val in df.label]
df = df.loc[np.repeat(df.index.values, reps)].reset_index(drop=True)
Balanced_train = df.to_numpy()
print('Shape before Balancing :', new_train_dataset.shape)
print('Shape After Balancing:', Balanced_train.shape)

Shape before Balancing : (55792, 601)
Shape After Balancing: (379134, 601)


Logistic Regression After Balancing

In [34]:
reg = LogisticRegression()
reg.fit(Balanced_train[:,:-1],Balanced_train[:,-1])

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

Logistic Regression Evaluation

In [35]:
print("score on test: "  + str(reg.score(x_test, y_test)))
print("score on train: " + str(reg.score(Balanced_train[:,:-1],Balanced_train[:,-1])))
y_pred = reg.predict(x_test)
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

score on test: 0.4984256659007744
score on train: 0.9042581250956127
              precision    recall  f1-score   support

           0       0.99      0.48      0.64     22403
           1       0.08      0.93      0.15      1099

    accuracy                           0.50     23502
   macro avg       0.54      0.70      0.40     23502
weighted avg       0.95      0.50      0.62     23502

[[10694 11709]
 [   79  1020]]


SVM After Balancing

In [36]:
svm=LinearSVC(C=0.0001)
svm.fit(Balanced_train[:,:-1],Balanced_train[:,-1])


LinearSVC(C=0.0001, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0)

SVM Evaluation

In [37]:
print("score on test: " + str(svm.score(x_test, y_test)))
print("score on train: "+ str(svm.score(Balanced_train[:,:-1], Balanced_train[:,-1])))
y_pred = svm.predict(x_test)
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

score on test: 0.2675942472981023
score on train: 0.8843680598416391
              precision    recall  f1-score   support

           0       0.99      0.23      0.38     22403
           1       0.06      0.97      0.11      1099

    accuracy                           0.27     23502
   macro avg       0.53      0.60      0.24     23502
weighted avg       0.95      0.27      0.37     23502

[[ 5219 17184]
 [   29  1070]]


Balancing the dataset for KNN model

In [38]:
new_train_dataset = np.concatenate((train_inputs,train_targets.reshape((train_targets.shape[0],1))),axis=1)
df = pd.DataFrame(new_train_dataset, columns = None)
df.columns = [*df.columns[:-1], 'label']
reps = [64 if val == 1 else 1 for val in df.label]
df = df.loc[np.repeat(df.index.values, reps)].reset_index(drop=True)
Balanced_train = df.to_numpy()
print('Shape before Balancing :', new_train_dataset.shape)
print('Shape After Balancing:', Balanced_train.shape)

Shape before Balancing : (55792, 601)
Shape After Balancing: (216190, 601)


Balance KNN implementation

In [39]:
knn = KNeighborsClassifier(n_neighbors=4)
knn.fit(Balanced_train[:,:-1],Balanced_train[:,-1])

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=4, p=2,
                     weights='uniform')

Balanced KNN evaluation

In [40]:
y_pred = knn.predict(x_test)
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99     22403
           1       0.73      0.74      0.73      1099

    accuracy                           0.97     23502
   macro avg       0.86      0.86      0.86     23502
weighted avg       0.97      0.97      0.97     23502

[[22100   303]
 [  289   810]]


Models' Summary

In [76]:
conclusion = pd.DataFrame([['Neural Network',precision_1,recall_1,f1Score_1],
              ['Logistic Regression',0.53,0.17,0.26],
              ['SVM',0.38,0.01,0.03],
              ['KNN',0.94,0.56,0.70],
              ['Neural Network after Balancing',precision_2,recall_2,f1Score_2],
              ['Logistic Regression after Balancing',0.08,0.93,0.15],
              ['SVM after Balancing',0.06,0.97,0.11],
              ['KNN after Balancing',0.73,0.73,0.73]],
              columns=["Algorithm","Precision","Recall","F1score"])
conclusion = conclusion.set_index('Algorithm')
conclusion.style.background_gradient(cmap="YlOrRd")


,Precision,Recall,F1score
Algorithm,,,
Neural Network,0.980000,0.330000,0.490000
Logistic Regression,0.530000,0.170000,0.260000
SVM,0.380000,0.010000,0.030000
KNN,0.940000,0.560000,0.700000
Neural Network after Balancing,0.880000,0.600000,0.710000
Logistic Regression after Balancing,0.080000,0.930000,0.150000
SVM after Balancing,0.060000,0.970000,0.110000
KNN after Balancing,0.730000,0.730000,0.730000


As you see After Blancing the dataset Neural Network and KNN model do the best !